In [2]:
%pip install requests beautifulsoup4 -q
%pip install watson-developer-cloud==1.5 -q
%pip install -U langchain-cli


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
import os
import json

In [17]:
load_dotenv()
api_key = os.getenv("WATSONX_API_KEY", None)
ibm_cloud_url = os.getenv("WATSONX_MODEL_ENDPOINT", None)
project_id = os.getenv("PROJECT_ID", None)
openai_api_key = os.getenv("OPENAI_API_KEY", None)
if api_key is None or ibm_cloud_url is None or project_id is None:
    print("Ensure you copied the .env file that you created earlier into the same directory as this notebook")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

model_params = {
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MIN_NEW_TOKENS: 30,
        GenParams.MAX_NEW_TOKENS: 2000,
        GenParams.RANDOM_SEED: None,
        GenParams.TEMPERATURE: 0,
        GenParams.REPETITION_PENALTY: 1.05
    }


    # Instantiate a model proxy object to send your requests
llm = Model(
    model_id='meta-llama/llama-3-70b-instruct',
    params=model_params,
    credentials=creds,
    project_id=project_id)

llm_to_langchain=llm.to_langchain()



In [10]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import re

def extract_text_from_website(url):
    # Fetch the HTML content of the webpage
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract all text from the webpage
        raw_text = soup.get_text()
        
        # Remove excess whitespaces and newlines
        cleaned_text = re.sub(r'\s+', ' ', raw_text).strip()
        
        # Remove HTML tags
        cleaned_text = re.sub(r'<.*?>', '', cleaned_text)
        
        return cleaned_text
    else:
        # If request was unsuccessful, return None
        print("Failed to retrieve webpage:", response.status_code)
        return None

def scrape_multiple_websites(urls):
    with ThreadPoolExecutor() as executor:
        # Submit each URL for scraping concurrently
        futures = [executor.submit(extract_text_from_website, url) for url in urls]
        # Retrieve results as they become available
        results = [future.result() for future in futures]
        
    # Store text from each URL as a separate item in a list
    return results



In [11]:
# Example usage
urls = [
    "https://ai.meta.com/blog/code-llama-large-language-model-coding/", 
    "https://huggingface.co/deepseek-ai/deepseek-coder-33b-instruct",
    "https://huggingface.co/tiiuae/falcon-180B",
    "https://huggingface.co/tiiuae/falcon-40b",
    "https://huggingface.co/google/flan-t5-xl",
    "https://huggingface.co/google/flan-t5-xxl",
    "https://huggingface.co/google/flan-ul2",
    "https://huggingface.co/core42/jais-13b-chat",
    "https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-instruct",
    "https://huggingface.co/meta-llama/Llama-2-13b-hf",
    "https://huggingface.co/mncai/llama2-13b-dpo-v7",
    "https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2",
    "https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1",
    "https://huggingface.co/TheBloke/Mixtral-8x7B-v0.1-GPTQ",
    "https://huggingface.co/intfloat/multilingual-e5-large",
    "https://huggingface.co/kaist-ai/prometheus-13b-v1.0",
    "https://www.ibm.com/docs/en/cloud-paks/cp-data/4.8.x?topic=models-granite-13b-v1-model-card",
    "https://www.ibm.com/docs/en/cloud-paks/cp-data/4.8.x?topic=models-granite-13b-instruct-v2-model-card",
    "https://www.ibm.com/docs/en/cloud-paks/cp-data/4.8.x?topic=models-granite-13b-chat-v2-model-card"
]


In [12]:
def prompt_to_llm(prompt_template, **variables):
    return llm_to_langchain.invoke(prompt_template.format(**variables))

In [13]:
model_row_prompt = '''
[INST] 
You are a helpful code assistant. Your task is to generate a valid JSON object based on the given information: {cleaned_model_data}.  
Use the following schema: {json_schema}.
Here is an example: {json_example}.
Just generate the JSON object without explanations:
[/INST] 
'''

In [14]:
json_schema = {
    "ModelName": {
      "type": "string"
    },
    "Size": {
      "type": "string"
    },
    "ID": {
      "type": "string"
    },
    "Provider": {
      "type": "string"
    },
    "Architecture": {
      "type": "string"
    },
    "ContextLength": {
      "type": "integer"
    },
    "Price": {
      "type": "number"
    },
    "Languages": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "TunningInformation": {
      "type": "string"
    },
    "TrainingData": {
      "type": "string"
    },
    "UsesSupported": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "OptimisedFor": {
      "type": "string"
    },
    "PromptingAdvice": {
      "type": "string"
    },
    "Output": {
      "type": "string"
    },
    "PromptTuningAvailability": {
      "type": "boolean"
    },
    "RegionalAvailability": {
      "type": "string"
    },
    "License": {
      "type": "string"
    }
  }

In [15]:
json_example = '{\n  "ModelName": "Code Llama",\n  "Size": "70B",\n  "ID": "CodeLlama - 70B",\n  "Provider": "Meta",\n  "Architecture": "Decoder-only",\n  "ContextLength": 100000,\n  "Price": "Free",\n  "Languages": ["Python", "C++", "Java", "PHP", "Typescript (Javascript)", "C#", "Bash"],\n  "TunningInformation": Instruction-tuned,\n  "TrainingData": "Code and code-related data",\n  "UsesSupported": ["Code generation", "natural language about code", "debugging"],\n  "OptimisedFor": "Low latency, real-time code completion, long inputs",\n  "PromptingAdvice": "Provide the model with code or natural language prompts",\n  "Output": "Code and natural language about code",\n  "PromptTuningAvailability": true,\n  "RegionalAvailability": "Not specified",\n  "License": "Community license as Llama 2"\n}'

In [19]:
cleaned_models_data = []
for url in urls:
    cleaned_model_data = extract_text_from_website(url)
    cleaned_models_data.append(cleaned_model_data)
    print(cleaned_model_data)

Introducing Code Llama, a state-of-the-art large language model for coding Our approachResearchMeta AIMeta LlamaBlogTry Meta AIFEATUREDLarge Language Model Introducing Code Llama, a state-of-the-art large language model for codingAugust 24, 2023TakeawaysUpdate: Jan 29, 2024: Releasing Code Llama 70BWe are releasing Code Llama 70B, the largest and best-performing model in the Code Llama familyCode Llama 70B is available in the same three versions as previously released Code Llama models, all free for research and commercial use:CodeLlama - 70B, the foundational code model;CodeLlama - 70B - Python, 70B specialized for Python;and Code Llama - 70B - Instruct 70B, which is fine-tuned for understanding natural language instructions.Code Llama is a state-of-the-art LLM capable of generating code, and natural language about code, from both code and natural language prompts.Code Llama is free for research and commercial use.Code Llama is built on top of Llama 2 and is available in three models:

In [ ]:
model_rows = []
for cleaned_model_data in cleaned_models_data:
    model_csv_row = prompt_to_llm(model_row_prompt, cleaned_model_data=cleaned_model_data, json_example=json_example, json_schema=json_schema)
    model_rows.append(model_csv_row)

In [ ]:
model_rows

['Here is the JSON file created using the provided information:\n\n```json\n{\n  "ModelName": "Code Llama",\n  "Size": "70B",\n  "ID": "CodeLlama - 70B",\n  "Provider": "Meta",\n  "Architecture": "Decoder-only",\n  "ContextLength": 100000,\n  "Price": "Free",\n  "Languages": ["Python", "C++", "Java", "PHP", "Typescript (Javascript)", "C#", "Bash"],\n  "TunningInformation": "Instruction-tuned",\n  "TrainingData": "Code and code-related data",\n  "UsesSupported": ["Code generation", "natural language about code", "debugging"],\n  "OptimisedFor": "Low latency, real-time code completion, long inputs",\n  "PromptingAdvice": "Provide the model with code or natural language prompts",\n  "Output": "Code and natural language about code",\n  "PromptTuningAvailability": true,\n  "RegionalAvailability": "Not specified",\n  "License": "Community license as Llama 2"\n}\n```\nPlease note that the `TunningInformation` field is a string and I have added quotes around it.',
 'Here is the JSON file creat

In [ ]:
import re

def delete_before_curly_bracket(input_string):
    # Define a regular expression pattern to match everything before the curly bracket
    pattern = re.compile(r'.*?({.*)', re.DOTALL)
    
    # Use the pattern to find the matching substring
    match = pattern.search(input_string)
    
    if match:
        # Return the substring starting from the curly bracket
        return match.group(1)
    else:
        # If no match is found, return the original string
        return input_string
    
def delete_after_last_curly_bracket(input_string):
    # Define a regular expression pattern to match everything after the last curly bracket
    pattern = re.compile(r'(.*})(?:.*)', re.DOTALL)
    
    # Use the pattern to find the matching substring
    match = pattern.search(input_string)
    
    if match:
        # Return the substring up to the last curly bracket
        return match.group(1)
    else:
        # If no match is found, return the original string
        return input_string
    
def delete_before_modelname(input_string):
    # Define a regular expression pattern to match everything before the curly bracket
    pattern = re.compile(r'.*?({.*)', re.DOTALL)
    
    # Use the pattern to find the matching substring
    match = pattern.search(input_string)
    
    if match:
        # Return the substring starting from the curly bracket
        return match.group(1)
    else:
        # If no match is found, return the original string
        return input_string


In [ ]:
clean_json = []
for row in model_rows:
    clean_row = delete_before_curly_bracket(row)
    final_row = delete_after_last_curly_bracket(clean_row)
    clean_json.append(final_row)

In [5]:
clean_json

NameError: name 'clean_json' is not defined

In [ ]:
finished_json = []
for row in clean_json:
    formated_row = json.dumps(row)
    finished_json.append(formated_row)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
# List where each item is a valid JSON object representing a model
finished_json

NameError: name 'finished_json' is not defined

In [ ]:
json_data = json.dumps(clean_json)


In [ ]:
# JSON string with all of the models
json_data

'["{\\n  \\"ModelName\\": \\"Code Llama\\",\\n  \\"Size\\": \\"70B\\",\\n  \\"ID\\": \\"CodeLlama - 70B\\",\\n  \\"Provider\\": \\"Meta\\",\\n  \\"Architecture\\": \\"Decoder-only\\",\\n  \\"ContextLength\\": 100000,\\n  \\"Price\\": \\"Free\\",\\n  \\"Languages\\": [\\"Python\\", \\"C++\\", \\"Java\\", \\"PHP\\", \\"Typescript (Javascript)\\", \\"C#\\", \\"Bash\\"],\\n  \\"TunningInformation\\": \\"Instruction-tuned\\",\\n  \\"TrainingData\\": \\"Code and code-related data\\",\\n  \\"UsesSupported\\": [\\"Code generation\\", \\"natural language about code\\", \\"debugging\\"],\\n  \\"OptimisedFor\\": \\"Low latency, real-time code completion, long inputs\\",\\n  \\"PromptingAdvice\\": \\"Provide the model with code or natural language prompts\\",\\n  \\"Output\\": \\"Code and natural language about code\\",\\n  \\"PromptTuningAvailability\\": true,\\n  \\"RegionalAvailability\\": \\"Not specified\\",\\n  \\"License\\": \\"Community license as Llama 2\\"\\n}", "{\\n  \\"ModelName\\": \